# Timer

In [14]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 22, 13, 30

while True:
    dt = datetime.now()
    if dt.month >= month and dt.day >= day:
        if dt.hour > hour or (dt.hour == hour and dt.minute > minute):
            break
    sleep(60)

# Parameter optimization

In [15]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['PriceChange']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 50
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'PriceChange': {"low_price_quantile": [1 * (i + 1) for i in range(20)]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['indicator_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat = stat.sort_values(['indicator_rank', 'pct_right_forecast_avg'], ascending=False)
stat.head(20)

Number of combinations is 20


  0%|          | 0/20 [00:00<?, ?it/s]

,pattern,PriceChange_low_price_quantile,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,indicator_rank
0,PriceChange,1,100.0,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,4.47,6.52,0.57,3.03,2.05,8.20,8.65,4.72,5.33,8.04,7.63,12.51,18.08,19.31,20.71,17.88,18.82,20.34,24.68,24.23,24.19,20.30,23.94,26.12,1,100.000000,30.000000
1,PriceChange,2,100.0,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,4.47,6.52,0.57,3.03,2.05,8.20,8.65,4.72,5.33,8.04,7.63,12.51,18.08,19.31,20.71,17.88,18.82,20.34,24.68,24.23,24.19,20.30,23.94,26.12,1,100.000000,30.000000
2,PriceChange,3,100.0,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,4.47,6.52,0.57,3.03,2.05,8.20,8.65,4.72,5.33,8.04,7.63,12.51,18.08,19.31,20.71,17.88,18.82,20.34,24.68,24.23,24.19,20.30,23.94,26.12,1,100.000000,30.000000
3,PriceChange,4,100.0,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,4.47,6.52,0.57,3.03,2.05,8.20,8.65,4.72,5.33,8.04,7.63,12.51,18.08,19.31,20.71,17.88,18.82,20.34,24.68,24.23,24.19,20.30,23.94,26.12,1,100.000000,30.000000
4,PriceChange,5,100.0,100.00,100.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,100.00,100.00,100.00,100.00,100.00,50.00,100.00,100.00,100.00,100.00,100.00,50.00,100.00,3.03,3.98,0.65,1.22,0.63,3.52,4.14,1.69,1.72,3.18,3.54,6.44,9.20,10.34,10.75,9.30,9.27,10.34,12.72,12.36,12.30,10.32,11.82,13.13,2,79.166667,18.333333
5,PriceChange,6,100.0,100.00,100.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,100.00,100.00,100.00,100.00,100.00,50.00,100.00,100.00,100.00,100.00,100.00,50.00,100.00,3.03,3.98,0.65,1.22,0.63,3.52,4.14,1.69,1.72,3.18,3.54,6.44,9.20,10.34,10.75,9.30,9.27,10.34,12.72,12.36,12.30,10.32,11.82,13.13,2,79.166667,18.333333
7,PriceChange,8,100.0,75.00,75.00,50.00,75.00,75.00,50.00,50.00,50.00,50.00,50.00,75.00,75.00,75.00,75.00,75.00,50.00,75.00,75.00,75.00,75.00,75.00,50.00,75.00,3.03,3.98,0.65,1.46,1.08,4.33,4.33,2.01,2.20,3.56,3.54,6.44,9.20,10.34,10.75,9.30,9.27,10.34,12.72,12.36,12.30,10.32,11.82,13.13,4,67.708333,-9.166667
15,PriceChange,16,100.0,62.50,62.50,50.00,62.50,62.50,50.00,50.00,50.00,50.00,62.50,75.00,75.00,75.00,75.00,75.00,62.50,75.00,75.00,75.00,75.00,75.00,62.50,75.00,2.79,3.64,0.65,1.46,1.08,3.13,4.33,2.01,2.20,3.73,5.19,8.48,11.80,12.04,13.59,10.76,11.08,11.25,13.48,12.36,12.40,10.42,12.48,13.96,8,67.187500,-22.500000
6,PriceChange,7,100.0,66.67,66.67,33.33,66.67,66.67,33.33,33.33,33.33,33.33,33.33,66.67,66.67,66.67,66.67,66.67,33.33,66.67,66.67,66.67,66.67,66.67,33.33,66.67,1.58,1.44,0.57,-0.12,0.12,0.46,0.00,-0.69,-0.92,-0.92,-0.55,0.38,0.31,1.38,0.79,0.72,-0.28,0.34,0.76,0.48,0.41,0.34,-0.31,0.14,3,56.945417,-39.163750
11,PriceChange,12,100.0,

# Save new config data to config file

In [13]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [12]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)